In [ ]:
import pandas as pd
import numpy as np
import itertools as it
import matplotlib.pyplot as plt
from scipy import stats
from collections import defaultdict

In [43]:
players = pd.read_csv('./data/players.csv')
games = pd.read_csv('./data/games.csv')
GAME1 = games.iloc[0].gameId

In [7]:
plays = pd.read_csv('./data/plays.csv')
p1 = plays[plays['gameId'] == GAME1]
week1 = pd.read_csv('./data/week1.csv')
week1 = week1.loc[(week1['gameId'] == GAME1)]
pd.unique(week1.event)

array(['None', 'ball_snap', 'pass_forward', 'pass_arrived',
       'pass_outcome_caught', 'out_of_bounds', 'pass_outcome_incomplete',
       'first_contact', 'tackle', 'man_in_motion', 'play_action',
       'qb_sack', 'fumble', 'fumble_offense_recovered', 'handoff',
       'pass_tipped', 'pass_outcome_interception', 'qb_strip_sack',
       'pass_shovel', 'line_set'], dtype=object)

In [23]:
p1 = p1[p1.passResult.isin(['C'])]

In [30]:
def mapIdToName(id):
    return players[players['nflId'] == id].iloc[0].displayName

In [27]:
def getClosest(players, football):
    distances = []
    for index, row in players.iterrows():
        distance = np.linalg.norm(np.subtract([football.x.values[0], football.y.values[0]], [row.x, row.y]))
        distances.append((row.nflId, distance))
    closest = sorted(distances, key= lambda t: t[1])[0][0]
    return closest

In [28]:
tackle_acc = defaultdict(list)
total_tackles = defaultdict(int)

In [41]:

        
for PLAY in p1.playId:
    curr_play = week1[week1['playId'] == PLAY]
    DEFENSIVE_TEAM = curr_play[curr_play['position'] == 'CB'].team.values[0]
    
    play_events = pd.unique(curr_play.event)
    if 'fumble' in play_events:
        print('Fumble play:', PLAY)
    elif 'first_contact' in play_events:
        
        first_contact = curr_play[curr_play['event'] == 'first_contact']
        football = first_contact[first_contact['team'] == 'football']
        defensive_players = first_contact[first_contact['team'] == DEFENSIVE_TEAM]
        closest1 = getClosest(defensive_players, football)
        
        if 'tackle' in play_events:
            END_EVENT = 'tackle'
        elif 'out_of_bounds' in play_events:
            END_EVENT = 'out_of_bounds'
        else:
            continue
    
        end = curr_play[curr_play['event'] == END_EVENT]
        football = end[end['team'] == 'football']
        defensive_players = end[end['team'] == DEFENSIVE_TEAM]
        closest2 = getClosest(defensive_players, football)
        
#         print(closest1, closest2, END_EVENT)
        total_tackles[closest2] += 1
        if closest1 == closest2:
            tackle_acc[closest1].append(1)
        else:
            tackle_acc[closest1].append(0)
dict(tackle_acc)  

{2552689.0: [1, 1, 1, 1],
 2539653.0: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 2495613.0: [0, 1, 0, 1, 0, 1, 0, 1],
 2555255.0: [1, 1, 1, 1],
 2555162.0: [0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1],
 79848.0: [1, 1, 1, 1, 1, 1, 1, 1],
 2556445.0: [0, 0, 0, 0],
 2539334.0: [1, 1, 1, 1],
 2552315.0: [0, 0, 0, 0],
 2543850.0: [0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1],
 2557967.0: [1, 1, 1, 1],
 2555383.0: [1, 0, 1, 0, 1, 0, 1, 0],
 2557958.0: [1, 1, 1, 1],
 2552301.0: [0, 0, 0, 0],
 2534832.0: [1, 0, 1, 0, 1, 0, 1, 0]}

In [42]:
tackle_acc_r = {mapIdToName(k): np.sum(tackle_acc[k]) / len(tackle_acc[k]) for k,v in tackle_acc.items()}
tackle_acc_r

{'Ronald Darby': 1.0,
 'Robert Alford': 1.0,
 'Corey Graham': 0.5,
 "De'Vondre Campbell": 1.0,
 'Deion Jones': 0.6666666666666666,
 'Malcolm Jenkins': 1.0,
 'Brian Poole': 0.0,
 'Desmond Trufant': 1.0,
 'Jordan Hicks': 0.0,
 'Ricardo Allen': 0.6666666666666666,
 'Duke Riley': 1.0,
 'Jalen Mills': 0.5,
 'Sidney Jones': 1.0,
 'Vic Beasley': 0.0,
 'Rodney McLeod': 0.5}

In [48]:
tackle_acc = defaultdict(list)
for week_num in range(1, 18):
    print(week_num)
    g_week = games[games['week'] == week_num]
    week_str = './data/week' + str(week_num) + '.csv'
    current_week = pd.read_csv(week_str)
    
    for g_ind, g_row in g_week.iterrows():
        GAMEID = g_row.gameId
        cwg = current_week[current_week['gameId'] == GAMEID]
        pg = plays[plays['gameId'] == GAMEID]
        
        for PLAY in pg.playId:
            curr_play = cwg[cwg['playId'] == PLAY]
            try:
                DEFENSIVE_TEAM = curr_play[curr_play['position'] == 'CB'].team.values[0]
            except:
                continue

            play_events = pd.unique(curr_play.event)
            if 'fumble' in play_events:
#                 print('Fumble play:', PLAY)
                pass
            elif 'first_contact' in play_events:
                try:
                    first_contact = curr_play[curr_play['event'] == 'first_contact']
                    football = first_contact[first_contact['team'] == 'football']
                    defensive_players = first_contact[first_contact['team'] == DEFENSIVE_TEAM]
                    closest1 = getClosest(defensive_players, football)

                    if 'tackle' in play_events:
                        END_EVENT = 'tackle'
                    elif 'out_of_bounds' in play_events:
                        END_EVENT = 'out_of_bounds'
                    else:
                        continue

                    end = curr_play[curr_play['event'] == END_EVENT]
                    football = end[end['team'] == 'football']
                    defensive_players = end[end['team'] == DEFENSIVE_TEAM]
                    closest2 = getClosest(defensive_players, football)

                    if closest1 == closest2:
                        tackle_acc[closest1].append(1)
                    else:
                        tackle_acc[closest1].append(0)
                except:
                    continue

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17


In [1]:
tackle_acc_r = {mapIdToName(k): np.sum(v) / len(v) for k,v in tackle_acc.items() if len(v) > 17}
sorted(tackle_acc_r.items(), key=lambda t: t[1], reverse=True)[:50]

NameError: name 'tackle_acc' is not defined